In [ ]:
## clear ram
# import torch
# with torch.no_grad():
#     torch.cuda.empty_cache()

In [7]:
### Load libraries

In [1]:
########## imports  ##############
import os
import time
import sys
import numpy as np
import scipy as sp
from scipy import integrate, interpolate
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import pickle
from torch.nn.functional import normalize

import warnings
warnings.filterwarnings('ignore')
# Path for various functions
module_path= os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import torch
import os
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
criterion = torch.nn.MSELoss(reduction='none')
import ResNet as net
from utils import *
from Autoencoder import *
from aHiTs import *
import lstmrnn
print('Libraries imported!')

Libraries imported!


In [2]:
# paths
noise = 0.0 
system = 'FHN'  
data_dir = os.path.join('../data/', system)
model_dir = os.path.join('../models/', system)
path_to_result=os.path.join('../results/', system)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
#### LOAD DATA

In [4]:
train_data = torch.load(data_dir + "/train_data.pt")
val_data = torch.load(data_dir + "/val_data.pt")
test_data = torch.load(data_dir + "/test_data.pt")

In [5]:
# load data
# train_data = np.load(os.path.join(data_dir, 'train_noise{}.npy'.format(noise)))
# val_data = np.load(os.path.join(data_dir, 'val_noise{}.npy'.format(noise)))
# test_data = np.load(os.path.join(data_dir, 'test_noise{}.npy'.format(noise)))
n_train = train_data.shape[0]
n_val = val_data.shape[0]
n_test = test_data.shape[0]
train_data=torch.tensor(train_data).to(torch.float32)
# train_data = train_data
# sets=train_data.shape[0]
# dataloader=torch.utils.data.DataLoader(x, batch_size=sets)
train_data.shape,test_data.shape,val_data.shape

(torch.Size([1000, 5121, 100]),
 torch.Size([10, 51200, 100]),
 torch.Size([10, 51200, 100]))

In [ ]:
#Params for AE
input_size=train_data.shape[2]
hidden_size=2 # select the appropriate size for the latent vector
L1=100 
L2=100
L3=100

class AE(nn.Module):
    def __init__(self, L1, L2, L3, input_size, hidden_size):
        super(AE, self).__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(input_size, L1),
            torch.nn.ReLU(),
            torch.nn.Linear(L1, L2),
            torch.nn.ReLU(),
            torch.nn.Linear(L2, L3),
            torch.nn.ReLU(),
            torch.nn.Linear(L3, hidden_size)
        )
        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # The Sigmoid activation function
        # outputs the value between 0 and 1
        # 9 ==> 784
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, L3),
            torch.nn.ReLU(),
            torch.nn.Linear(L3, L2),
            torch.nn.ReLU(),
            torch.nn.Linear(L2, L1),
            torch.nn.ReLU(),
            torch.nn.Linear(L1,input_size)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded
    
Autoencoder = AE(L1,L2,L3,input_size=input_size, hidden_size=hidden_size).to(device)


#Params for ResNet
tspan=encoded.shape[1]
np.random.seed(2)  # for reproduction
dt = 0.01  #0.001
train_steps = tspan  # at least equal to the largest step size
val_steps =tspan
test_steps = tspan  # t=20
t = np.linspace(0, (train_steps-1)*dt, train_steps)
datasets.append(DataSet(z_train, z_valid, z_test, dt, step_size=step_size, n_forward=5)

datasets = list()
step_sizes = list()
print('Dt\'s: ')
#a=[10,20,30,60,100,200,400,600,800]
for i in range(10,11):
    step_size = 2**i  #exponential function
    step_sizes.append(step_size)
    datasets.append(DataSet(z_train, z_valid, z_test, dt, step_size=step_size, n_forward=5))

models = list()
# max_epoch=20000
L1=128
L2=256
L3=512
L4=1024
L5=2048
z=hidden_size

for (step_size, dataset) in zip(step_sizes, datasets):
    # set up the network
    model_name = 'modelnew_D{}_noise{}.pt'.format(step_size,noise)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    ResNet = net.ResNet(arch=[z,L1,L1,L1,z], dt=dt, step_size=step_size)
    models.append(modelresnet)


In [ ]:
# Define loss functions
reconstruction_loss = nn.MSELoss()
time_stepping_loss = nn.MSELoss()

# Define optimizer
AE_RN = nn.ModuleList([Autoencoder(), ResNet()])
optimizer = optim.Adam(AE_RN.parameters(), lr=0.001)

# Training loop
num_epochs = 100
losses = []

for epoch in range(num_epochs):
    total_loss = 0.0
    for data in train_data:
        # Forward pass for AE
        z_train, train_reconstructed = AE_RN[0](data)
        z_valid, val_reconstructed = AE_RN[0](val_data)
        z_test, test_reconstructed = AE_RN[0](test_data)
        # prepare data for ResNet
        net.DataSet(encoded, z_valid, z_test, dt, step_size=step_size, n_forward=5)
        n_samples = dataset.n_train
        new_idxs = torch.randperm(n_samples)
        batch_x = dataset.train_x[new_idxs[:batch_size], :]
        batch_ys = dataset.train_ys[new_idxs[:batch_size], :, :]
        # =============== calculate losses ================
        train_loss = self.calculate_loss(batch_x, batch_ys, w=w)
        val_loss = self.calculate_loss(dataset.val_x, dataset.val_ys, w=w)
        
        def calculate_loss(self, x, ys, w=1.0):
        """
        :param x: x batch, array of size batch_size x n_dim
        :param ys: ys batch, array of size batch_size x n_steps x n_dim
        :return: overall loss
        """
        batch_size, n_steps, n_dim = ys.size()
        assert n_dim == self.n_dim
        
        
        # forward (recurrence)
        y_preds = torch.zeros(batch_size, n_steps, n_dim).float().to(self.device)
        y_prev = x
        for t in range(n_steps):
            y_next = self.forward(y_prev)
            y_preds[:, t, :] = y_next
            y_prev = y_next

        # compute loss
        criterion = torch.nn.MSELoss(reduction='none')
        loss = w * criterion(y_preds, ys).mean() + (1-w) * criterion(y_preds, ys).max()

        return loss
        
        
        
        estimated = AE_RN[1](latent)
        
        # Compute losses
        reconstruction_loss_value = reconstruction_loss(data, reconstructed)# AE loss
        time_stepping_loss_value = time_stepping_loss(latent, estimated)
        total_loss = reconstruction_loss_value + time_estimation_loss_value
        
        # Backpropagation and optimization
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
    
    # Print training progress
    print(f"Epoch: {epoch+1}/{num_epochs}, Reconstruction Loss: {reconstruction_loss_value.item()}, Time Estimation Loss: {time_estimation_loss_value.item()}")
    losses.append(total_loss.item())

# Plot the combined loss function
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Combined Loss Function')
plt.show()

# After training, test the model
with torch.no_grad():
    test_latent, test_reconstructed = ae_resnet[0](test_data)
    test_estimated = ae_resnet[1](test_latent)

    # Print reconstruction loss and time estimation loss for testing
    reconstruction_test_loss = reconstruction_loss(test_data, test_reconstructed)
    time_estimation_test_loss = time_estimation_loss(test_latent, test_estimated)
    print(f"Testing Reconstruction Loss: {reconstruction_test_loss.item()}")
    print(f"Testing Time Estimation Loss: {time_estimation_test_loss.item()}")
